# Variational Quantum Linear Solver

This demo shows how to apply the Variational Quantum Eigen-Solver (VQE) method to solve the following linear equation $ A x = b $:

$$
\begin{array}{ll}
\begin{bmatrix}
   2 &  5 & -13 \\
   3 & -9 &   3 \\
  -5 &  6 &   8 \\
\end{bmatrix} \begin{bmatrix}
  12 \\
  5 \\
  3 \\
\end{bmatrix} = \begin{bmatrix}
  10 \\
  0 \\
  -6 \\
\end{bmatrix}
\end{array}
$$

Note that compared with the original problem, we've divided by 100 at both side. This is harmless since they are equivalent.

### Step 1: Pre-process

First, quantum circuits are capable to handle matrix whose dimension is power of 2, we need to expand the equation from 3x3 to 4x4.  
To be specific, we add a constant 1 to $ A $, $ x $ and $ b $, calling it **scaling indicator** as we'll use it to decode the desired solution from a normalized quantum state later.  

$$
\begin{array}{ll}
\begin{bmatrix}
   2 &  5 & -13 & 0 \\
   3 & -9 &   3 & 0 \\
  -5 &  6 &   8 & 0 \\
   0 &  0 &   0 & 1 \\
\end{bmatrix} \begin{bmatrix}
  12 \\
  5 \\
  3 \\
  1 \\
\end{bmatrix} = \begin{bmatrix}
  10 \\
  0 \\
  -6 \\
  1 \\
\end{bmatrix}
\end{array}
$$

Second, following the VQE procedure, we'll encode the right-hand vector b to a quantum state $ \left| b \right> $, hence divide both side by $ ||b|| $, turning this equation into:

$$
\begin{array}{ll}
\begin{bmatrix}
   0.1709 &  0.4272 & -1.1107 & 0.     \\
   0.2563 & -0.7689 &  0.2563 & 0.     \\
  -0.4272 &  0.5126 &  0.6835 & 0.     \\
   0.     &  0.     &  0.     & 0.0854 \\
\end{bmatrix} \begin{bmatrix}
  12 \\
  5 \\
  3 \\
  1 \\
\end{bmatrix} = \begin{bmatrix}
  0.8544 \\
  0.     \\
 -0.5126 \\
  0.0854 \\
\end{bmatrix}
\end{array}
$$

Denote this equation as $ \tilde A \tilde x = \tilde b $.

### Step 3: VQE optimizing

To briefly explain VQE, one need to design a Hamiltonian H and a parametrized ansatz $ U(\theta) $, minimizing the expection $ E = \left< x | H | x \right> $,   
when $ E $ reaches the minimal, the ansatz-state $ \left| x \right> = U(\theta) \left| 0 \right> $ will just be the desired $ \left| \hat x \right> $, and the expectation will also be the energy value.

⚪ Hamiltonian design

Following essay [arXiv:1909.03898](https://arxiv.org/abs/1909.03898), we define the Hamiltonian $ H_A = A^\dagger (I - \left| b \right> \left< b \right|) A $, and one can easily verify that the normalized state $ \left| \hat x \right> = \frac{\tilde x}{||\tilde x||} $ is right the ground-state of $ H_A $ with energy 0.

⚪ Ansatz design

Then we're to design a parametrized ansatz to prepare this state $ \left| \hat x \right> $. And we found this circuit consisting of three RY gates and one CNOT gate will just do it:

```
--RY--o--RY--
      |
--RY--x------
```

This circuit only has 3 trainable parameters in RY gates, we zero-initialize all the parameters, and finally got [-0.047208991809220814, 0.7808882554867012, 0.509390326201012] after training.  

We implemented this part using both PennyLane and SpinQit, please refer to [run_VALA.py](./run_VALA.py) and [run_VALA_spinqit.py](./run_VALA_spinqit.py), respectively.  
NOTE: the PennyLane implementation has slight better numerical accuracy than the SpinQit one, we'd prefer it.

### Step 4: Run the pretrained circuit

Once the trainable parameters are well-optimized, we can run the circuit to obtain the ansatz-state, which corresponds to the ground-state of the Hamiltonian, and also corresponds to the solution of our expanded linear equation.

In [11]:
from spinqit import get_basic_simulator, get_compiler, BasicSimulatorConfig

shots = 100000

compiler = get_compiler('qasm')
exe = compiler.compile('log/run_VALA.qasm', level=0)
engine = get_basic_simulator()
config = BasicSimulatorConfig()
config.configure_shots(shots)
result = engine.execute(exe, config)

keys = ['00', '01', '10', '11']
state = [it.real for it in result.states]
probs = [result.probabilities.get(key, 0.0) for key in keys]
print('[Theoretical]')
print('state:', state)
print('probs:', probs)
counts = [result.counts.get(key, 0) for key in keys]
freqs = [result.counts.get(key, 0) / shots for key in keys]
print('[Practical]')
print('counts:', counts)
print('freqs:', freqs)

[Theoretical]
state: [0.8969221113267619, 0.37371754633444176, 0.22423052780399913, 0.07474350925430306]
probs: [0.8044692737868563, 0.13966480443823562, 0.05027932959926003, 0.005586592175648088]
[Practical]
counts: [80447, 13966, 5028, 559]
freqs: [0.80447, 0.13966, 0.05028, 0.00559]


### Step 5: Post-process

In order to decode the desired solution from the measured quantum results above, we need some additional classical operations.  
Firstly, we define several utility functions, and set up the ground truth for precision evaluations.  

In [12]:
import numpy as np
from numpy import ndarray

def state_norm(psi:ndarray) -> ndarray:
  return psi / np.linalg.norm(psi)

def get_fidelity(psi:ndarray, phi:ndarray) -> float:
  return np.abs(np.dot(psi.conj().T, phi)).item()

def get_L1_error(x:ndarray, y:ndarray) -> float:
  return np.abs(x - y).mean().item()

def decode(x_tilde:ndarray) -> ndarray:
  global x_gt
  x_hat = x_tilde / x_tilde[-1].item()
  x_hat = x_hat[:len(x_gt)]
  return x_hat

# The algebraic target solution
x_gt = np.asarray([12, 5, 3])
# The expanded and normalized target solution, corresponding to the ansatz-state
x_state_gt = state_norm(np.asarray([12, 5, 3, 1]))

print('The algebraic solution truth:', x_gt)
print('The quantum state truth:', x_state_gt)

The algebraic solution truth: [12  5  3]
The quantum state truth: [0.89692211 0.37371755 0.22423053 0.07474351]


⚪ Simulator

For simulators, we can access `state` and `probs`, and get a very ideal solution :)  
The fidelity just reaches 1.0 at the quantum side, and the L1 error is less than 1e-8 at the classical side.  

In [13]:
print('The quantum solution:', state)
print('>> fidelity:', get_fidelity(np.asarray(state), x_state_gt))

x_hat = decode(np.asarray(state))
print('The decoded solution:', x_hat)
print('>> L1 error:', get_L1_error(x_hat, x_gt))

The quantum solution: [0.8969221113267619, 0.37371754633444176, 0.22423052780399913, 0.07474350925430306]
>> fidelity: 1.0
The decoded solution: [12.  5.  3.]
>> L1 error: 1.6908560477683447e-09


⚪ Real-Chip

For real-hardware devices, we can only access `counts` and `freqs`, hence get an approximated solution :(  
Here we use `shots=10000`, hence the theoretical precision of `fidelity` will be `1e-5`, though in practice it reaches a better `1e-9` here. And the decoded solution shows a magnitude of `1e-3` in L1 error, which is just acceptible.  

In [14]:
# approximate the amplitude from probabilities
x_state_approx = np.sqrt(freqs)
print('The quantum solution (approx):', x_state_approx)
print('>> fidelity:', get_fidelity(x_state_approx, x_state_gt))

x_hat = decode(np.asarray(x_state_approx))
print('The decoded solution:', x_hat)
print('>> L1 error:', get_L1_error(x_hat, x_gt))

The quantum solution (approx): [0.89692252 0.37371112 0.22423202 0.0747663 ]
>> fidelity: 0.999999999718374
The decoded solution: [11.99634709  4.99838972  2.99910541]
>> L1 error: 0.002052592112998427
